In [1]:
import xarray as xr
import distributed

In [2]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57661,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.34 GiB
Comm: tcp://127.0.0.1:57690,Total threads: 1
Dashboard: http://127.0.0.1:57691/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:57664,


# Split ERA5 into training / test

In [3]:
ds_era5 = xr.open_dataset('out.nc',  engine='netcdf4').chunk({'time': 2})

In [4]:
ds_era5 = ds_era5.rename({'lat': 'latitude', 'lon': 'longitude'})

In [5]:
len(ds_era5)

3

In [6]:
ds_era5_train = ds_era5[['t2m']]
ds_era5_test = ds_era5[['t2m']]

In [8]:
ds_era5_test

<xarray.Dataset>
Dimensions:    (latitude: 32, longitude: 64, time: 62)
Coordinates:
  * time       (time) datetime64[ns] 2023-12-01 ... 2023-12-31T12:00:00
  * latitude   (latitude) float64 -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(2, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-12 07:50:10 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [7]:
ds_era5_train.to_zarr(
    '../data/processed/era5/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)
ds_era5_test.to_zarr(
    '../data/processed/era5/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)